# Rating Prediction Project - Web Scraping

### Scraping the data from Flipkart and Amazon

#### Importing required libraries

In [ ]:
import selenium
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

#### Scraping Ratings and Reviews from Flipkart

In [ ]:
driver=webdriver.Chrome(r'C:\Users\dell\Downloads\chromedriver_win32\Chromedriver.exe') 
time.sleep(2)

In [ ]:
url = "https://www.flipkart.com/"
driver.get(url)
driver.find_element_by_xpath("//div[@class='_2QfC02']/button").click()

In [ ]:
srch_items = ['laptops', 'Phones','smart watches','Monitors']

In [ ]:
title = []
review_text = []
ratings = []

In [ ]:
def scrap():    
        for i in driver.find_elements_by_xpath("//div[@class='t-ZTKy']"):
            review_text.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']"):
            title.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']/../div"):
            ratings.append(i.text)
        return

In [ ]:
urls=[]

In [ ]:
for i in srch_items:
    
    #Locating search bar
    
    srchBar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input")
    srchBar.clear()
    srchBar.send_keys(i)
    
    #clicking on search button
    
    driver.find_element_by_xpath("//button[@class='L0Z3Pu']").click()
    time.sleep(3)
    page = []
    for i in driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a"):
        page.append(i.get_attribute('href'))
    for i in page[0:4]:
        driver.get(i)
        time.sleep(3)
        items = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
        for i in items:
            urls.append(i.get_attribute('href'))

In [ ]:
len(urls)

In [ ]:
for i in urls:
    driver.get(i)
    time.sleep(2)
    for _ in range(2):
        driver.execute_script("window.scrollBy(0,6000)")
        time.sleep(1)
        
        #clicking on all reviews
    
    try:
        btn=driver.find_element_by_xpath("//div[@class='_2c2kV-']/following::a")
        lnk = btn.get_attribute('href')
        driver.get(lnk)
        time.sleep(1)
    except NoSuchElementException:
        pass

    scrap()        
    try:
        n_page=driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a")
        np=[]
        for i in n_page:
            np.append(i.get_attribute('href'))
        for i in np[0:18]:
            driver.get(i)
            time.sleep(1)
            scrap()
    except: continue

In [ ]:
len(ratings), len(review_text), len(title)

In [ ]:
#creating dataframe

data = list(zip(title,review_text,ratings))
df2 = pd.DataFrame(data, columns = ["Review_Title","Reiew_Text","Ratings"])
df2

In [ ]:
#saving the data into .csv file

df2.to_csv("flipkrt1_reviews.csv")

### Scraping Ratings and Reviews from Amazon

In [ ]:
driver=webdriver.Chrome(r'C:\Users\dell\Downloads\chromedriver_win32\Chromedriver.exe') 
time.sleep(2)

In [ ]:
url = "https://www.amazon.in/"
driver.get(url)
time.sleep(3)

In [ ]:
srch_items = ['laptops','phones','headphones','smart watches', 'Professional Cameras', 'Printers', 'Monitors', 'Home theater', 'Router']

In [ ]:
title = []
review_text = []
ratings = []

In [ ]:
for i in srch_items:
    srchbar = driver.find_element_by_id("twotabsearchtextbox")
    srchbar.send_keys(i)
    driver.find_element_by_id("nav-search-submit-button").click()  #clicking on search button
    time.sleep(1)
    item_url = []
    for i in driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a"):
        item_url.append(i.get_attribute('href'))
    for i in item_url:
        driver.get(i)
        time.sleep(1)
        for _ in range(2):
            driver.execute_script("window.scrollBy(0,6000)")
            time.sleep(1)
        
        #clicking on all reviews
        
        try:
            driver.find_element_by_xpath("//a[@class='a-link-emphasis a-text-bold']").click()
        except:
            pass
        
        url_ = []
        try:
            two_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[4]/td[2]/a")
            url_.append(two_str.get_attribute('href'))
        except: pass
        try:
            three_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[3]/td[2]/a")
            url_.append(three_str.get_attribute('href'))
        except: pass
        try:
            one_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[5]/td[2]/a")
            url_.append(one_str.get_attribute('href'))
        except: pass
        try:
            fiv_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[1]/td[2]/a")
            url_.append(fiv_str.get_attribute('href'))
        except: pass
        try:
            four_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[2]/td[2]/a")
            url_.append(four_str.get_attribute('href'))
        except: pass
        
        for i in url_:
            driver.get(i)
            time.sleep(1)
            for i in range(10):
                ids = driver.find_elements_by_xpath("//div[@class='a-section a-spacing-none review-views celwidget']/div")
                comment_ids = []
                for i in ids:
                    comment_ids.append(i.get_attribute('id'))

                for x in comment_ids:
                    
                    #Extracting user ids on page
                    
                    try:
                        review_title = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[2]/a[2]/span[1]')
                        title.append(review_title.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        title.append('')

                    try:
                        rating = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[2]/a[1]/i/span[1]')
                        ratings.append(rating.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        ratings.append('')

                    try:
                        text = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[4]/span/span')
                        review_text.append(text.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        review_text.append('')
                try:
                    driver.find_element_by_xpath("//ul[@class='a-pagination']/li[2]/a").click()
                    time.sleep(3)
                except : break 

In [ ]:
len(ratings), len(title), len(review_text)

In [ ]:
#creating a dataframe

data = list(zip(title,review_text,ratings))
df = pd.DataFrame(data, columns = ["Review_title","Reiew_text","Ratings"])
df

In [ ]:
#saving into csv file

df.to_csv("amazon1_reviews.csv")

In [ ]:
#Loading the flipkart reviews file

flpkrt = pd.read_csv("flipkrt1_reviews.csv")
flpkrt

In [ ]:
#Loading the amazon reviews file

df1 = pd.read_csv("amazon1_reviews.csv")
df1

### Updating the Column Names to same Names in both files

In [ ]:
# Changing the column name in amazon file similar to flipkart file

df1.rename(columns = {'Review_title':'Review_Title'}, inplace = True)

In [ ]:
# Changing the column name in amazon file similar to flipkart file

df1.rename(columns = {'Reiew_text':'Review_Text'}, inplace = True)

In [ ]:
# Changing the column name in flipkart file

flpkrt.rename(columns = {'Reiew_Text':'Review_Text'}, inplace = True)

In [ ]:
# Dropping Unnamed: 0 in flipkart file

flpkrt.drop(columns = 'Unnamed: 0', inplace=True)

In [ ]:
# Dropping Unnamed: 0 in amazon file

df1.drop(columns = 'Unnamed: 0', inplace=True)

In [ ]:
#Concating both files

Ratings_Prediction = pd.concat([df1, flpkrt],ignore_index=True)
Ratings_Prediction

In [ ]:
Ratings_Prediction.to_csv("Ratings_Prediction_Scrapped_Data_File.csv")